# PRACTICA GUIADA: Introducción al proceso de Feature Selection en SciKit Learn

## 1. Introducción

El objetivo de esta práctica es introducir algunos de los métodos para la selección de features implementados en Scikit-Learn. Nos concentraremos en algunos de los más comunes. Tal y como se ha mencionado en la exposición teórica, existen muchos más métodos. Pueden consultar sus características en la [documentación oficial del Scikit-Learn dedicada al tema](http://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection).


## 2. "Familias" de técnicas y métodos de feature selection

Existen diferentes "familias" de técnicas de feature selection. Veamos.


### 2.1 Filter Methods

#### 2.1.1 Features con baja varianza

Un primer caso (casi trivial) sería la remoción de aquellos features o características que no aporten "información" a nuestro dataset. Una forma de lograr la información es a través de la varianza. 

Entonces, lo que haremos será remover todos aquellos features que tengan baja varianza (es decir, baja variabilidad). Esto haremos definiendo un `threshold` por debajo del cual eliminaremos los features.


In [1]:
import pandas as pd
from scipy import random
import numpy as np
from sklearn import datasets

In [2]:
# Importamos los datasets Iris y Boston, que ya conocemos:

iris = datasets.load_iris()
boston = datasets.load_boston()


In [3]:
# Empecemos a trabajar con el dataset Iris. Creamos un DataFrame con los datos:

df1 = pd.DataFrame(np.concatenate((iris.data, iris.target.reshape(-1,1)),1),
                  columns = iris.feature_names + ['species'])

* Si aplicásemos un `threshold = 0.5` la única variable que debería ser excluida sería `sepal_width`:

In [4]:
df1.apply(np.var)

sepal length (cm)    0.681122
sepal width (cm)     0.188713
petal length (cm)    3.095503
petal width (cm)     0.577133
species              0.666667
dtype: float64

In [5]:
from sklearn.feature_selection import VarianceThreshold

# Instanciamos la clase VarianceThreshold definiendo un threshold=0.5

fet_sel = VarianceThreshold(threshold=0.5)

# Fiteamos a fet_sel con nuestro df1:

fet_sel.fit(df1)

VarianceThreshold(threshold=0.5)

* Usando el método `get_support` podemos consultar un array booleano que marca las variables que no han sido excluidas.

In [6]:
fet_sel.get_support()

array([ True, False,  True,  True,  True])

In [7]:
# Podemos filtrar las columnas que no superen el umbral de varianza:

df1.columns[fet_sel.get_support()]

Index(['sepal length (cm)', 'petal length (cm)', 'petal width (cm)',
       'species'],
      dtype='object')

* Si quisiéramos generar un nuevo array con los datos, simplemente podemos llamar al método `.transform(X)`

In [8]:
df1_reduced = pd.DataFrame(fet_sel.transform(df1), columns = df1.columns[fet_sel.get_support()])
df1_reduced.head()

,sepal length (cm),petal length (cm),petal width (cm),species
0,5.1,1.4,0.2,0.0
1,4.9,1.4,0.2,0.0
2,4.7,1.3,0.2,0.0
3,4.6,1.5,0.2,0.0
4,5.0,1.4,0.2,0.0


#### 2.1.2 Feature selection univariada

Tal y como lo indica el nombre, estos métodos buscan seleccionar features, generalmente comparándolas con el target, y utilizando diferente pruebas univariadas. Existen diferentes métodos que realizan esto.

#### `SelectKBest`

El primero que veremos (en un contexto de regresión) es `SelectKBest`. Este método toma dos argumentos:

* `score_func`: una función que devuelve algún score entre $X$ y $Y$:

    * Para regresión: f_regression, mutual_info_regression
    * Para clasificación: f_classif, mutual_info_classif


* `k`: la cantidad de "mejores" features que serán seleccionadas

Para facilitar el desarrollo hemos generado una función que ejecuta los métodos para regresion, extrae los atributos e imprime los resultados. 

* La función toma un dataframe como input
* Luego instancia el método `SelectKBest` con los parámetro `score_func=f_regression` y `k=2`
* Realiza el `.fit` del método
* Guarda los resultados (`scores`, `pvalues`, `get_support` y las columnas) en un dataframe y lo retorna. 


In [9]:
from sklearn.feature_selection import SelectKBest, f_classif, f_regression

df2 = pd.DataFrame(np.concatenate((boston.data, boston.target.reshape(-1,1)),1),
                  columns = list(boston.feature_names) + ['price'])

def select_kbest_reg(data_frame, target, k=2):
    """
    Seleccionado K-Best features para regresión
    :param data_frame: Un dataframe con datos
    :param target: target en el dataframe
    :param k: cantidad deseada de features
    :devuelve un dataframe llamado feature_scores con los scores para cada feature
    """
    feat_selector = SelectKBest(f_regression, k=k)
    _ = feat_selector.fit(data_frame.drop(target, axis=1), data_frame[target])
    
    feat_scores = pd.DataFrame()
    feat_scores["F Score"] = feat_selector.scores_
    feat_scores["P Value"] = feat_selector.pvalues_
    feat_scores["Support"] = feat_selector.get_support()
    feat_scores["Attribute"] = data_frame.drop(target, axis=1).columns
    
    return feat_scores

kbest_feat = select_kbest_reg(df2, "price", k=4)
kbest_feat = kbest_feat.sort_values(["F Score", "P Value"], ascending=[False, False])
kbest_feat

,F Score,P Value,Support,Attribute
12,601.617871,5.081103e-88,True,LSTAT
5,471.846740,2.487229e-74,True,RM
10,175.105543,1.609509e-34,True,PTRATIO
2,153.954883,4.900260e-31,True,INDUS
9,141.761357,5.637734e-29,False,TAX
4,112.591480,7.065042e-24,False,NOX
0,89.486115,1.173987e-19,False,CRIM
8,85.914278,5.465933e-19,False,RAD
6,83.477459,1.569982e-18,False,AGE
1,75.257642,5.713584e-17,False,ZN


* Puede verse entonces que si quisiéramos seleccionar las mejores 4 variables estas serían `LSTAT`,`RM`, `PTRATIO` e `INDUS`.

* Hasta aquí solamente ordenamos e imprimimos los resultados del `SelectKBest` ¿Cómo podríamos hacer para realizar efectivamente la selección?

In [10]:
select = kbest_feat.loc[kbest_feat['Support'] == True,'Attribute']
df2_reduced = df2[select]
df2_reduced.head()

,LSTAT,RM,PTRATIO,INDUS
0,4.98,6.575,15.3,2.31
1,9.14,6.421,17.8,7.07
2,4.03,7.185,17.8,7.07
3,2.94,6.998,18.7,2.18
4,5.33,7.147,18.7,2.18


* Si quisiéramos trabajar con un problema de clasificación podríamos simplemente cambiar el `score_func` a alguna función de scoring adecuada: `f_classif`, `mutual_info_classif`.

#### `SelectKPercentile`


Este método selecciona features en base a los percentiles (definidos por el usuario) de los scores máximos.


In [11]:
import pandas as pd
from sklearn.feature_selection import SelectPercentile, f_regression

def select_percentile(data_frame, target, percentile=15):

    feat_selector = SelectPercentile(f_regression, percentile=percentile)
    _ = feat_selector.fit(data_frame.drop(target, axis=1), data_frame[target])
    
    feat_scores = pd.DataFrame()
    feat_scores["F Score"] = feat_selector.scores_
    feat_scores["P Value"] = feat_selector.pvalues_
    feat_scores["Support"] = feat_selector.get_support()
    feat_scores["Attribute"] = data_frame.drop(target, axis=1).columns
    
    return feat_scores

In [12]:
per_feat = select_percentile(df2, "price", percentile=20)
per_feat = per_feat.sort_values(["F Score", "P Value"], ascending=[False, False])
per_feat

,F Score,P Value,Support,Attribute
12,601.617871,5.081103e-88,True,LSTAT
5,471.846740,2.487229e-74,True,RM
10,175.105543,1.609509e-34,True,PTRATIO
2,153.954883,4.900260e-31,False,INDUS
9,141.761357,5.637734e-29,False,TAX
4,112.591480,7.065042e-24,False,NOX
0,89.486115,1.173987e-19,False,CRIM
8,85.914278,5.465933e-19,False,RAD
6,83.477459,1.569982e-18,False,AGE
1,75.257642,5.713584e-17,False,ZN


### 2.2 Wrapper Methods 

#### 2.2.1 Eliminación Recursiva de Features (RFE)

Este método toma como input un estimador externo para tratar de cuantificar el peso (la importancia) de cada feature. El método va eliminando features sucesivamente para ir quedándose con sets cada vez más pequeño de features.

En primera instancia, el método entrena el estimador sobre el total de features y se cuantifica la importancia de cada feature a través de algún atributo del tipo `coef_` o `feature_importance`. El feature menos importante  se elimina del set y se vuelve a entrenar con los restantes. El proceso se repite de forma recursiva hasta que se llega al número de features definido previamente.

El método `RFE()` toma los siguientes argumentos:

* `estimator`: el estimador sobre el cual se va a realizar la selección (podría ser cualquier método supervisado que tenga un `.fit` que provea como importancia de cada feature un método `coef_` o `feature_importance`)
* `n_features_to_select`: la cantidad de features que se busca seleccionar
* `steps`: la cantidad de features que se elimina en cada iteración (puede pasarse en relativo o en absoluto).

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(df2.drop('price',axis = 1),df2['price'])
    
scaler = StandardScaler()
    
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=5, step = 1)
_ = selector.fit(X_train, y_train)

scores = pd.DataFrame()
scores["Attribute Name"] = df2.drop('price',axis = 1).columns
scores["Ranking"] = selector.ranking_
scores["Support"] = selector.support_

print(scores)

   Attribute Name  Ranking  Support
0            CRIM        6    False
1              ZN        5    False
2           INDUS        9    False
3            CHAS        7    False
4             NOX        1     True
5              RM        1     True
6             AGE        8    False
7             DIS        1     True
8             RAD        3    False
9             TAX        2    False
10        PTRATIO        1     True
11              B        4    False
12          LSTAT        1     True


* Ahora bien... ¿cuántos features tenemos que seleccionar? Ese es un problema que podemos resolver con... CrossValidation. 

#### 2.2.2 Eliminación Recursiva de Features con Cross Validation (RFECV)

Por suerte, hay un método que nos permite seleccionar mediante CrossValidation la cantidad de features a retener.

El método `RFECV()` toma los siguientes argumentos:
   * `estimator`: análogo a `RFE`, un estimador de aprendizaje supervisado con un atributo `coef_` o `feature_selection` 
   * `steps`: la cantidad de features que se elimina en cada iteración (puede pasarse en relativo o en absoluto).
   * `cv`: determina el método de valdiación cruzada. Puede pasarse un iterador.

In [15]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import KFold

In [16]:
kf = KFold(n_splits=5, shuffle = True)
estimator = LinearRegression()
selector = RFECV(estimator, step = 1, cv=kf, scoring = 'neg_mean_squared_error', verbose=2)
selector.fit(X_train, y_train)

Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting 

RFECV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
      estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                 normalize=False),
      min_features_to_select=1, n_jobs=None, scoring='neg_mean_squared_error',
      step=1, verbose=2)

* Podemos conocer cuáles son las variables seleccionadas luego del proceso de validación cruzada:

In [17]:
df2.drop('price', axis=1).loc[:,selector.support_].columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT'],
      dtype='object')

## 3. Feature Selection como parte de un pipeline



La introducción de un proceso de selección de features dentro de un pipeline es directa:

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [19]:
kf = KFold(n_splits=3, shuffle = True)
estim = LinearRegression()
select = RFE(estim, step = 1, verbose = 1)

pipe = Pipeline([
  ('feat_sel', select),
  ('reg', estim)])

param_grid = {'feat_sel__n_features_to_select' : np.arange(1,len(df2.columns)) }

gridcv = GridSearchCV(pipe, param_grid, cv=kf, verbose=1, n_jobs = 3)

gridcv.fit(X_train, y_train)

Fitting 3 folds for each of 13 candidates, totalling 39 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  34 out of  39 | elapsed:    1.8s remaining:    0.3s
[Parallel(n_jobs=3)]: Done  39 out of  39 | elapsed:    1.8s finished


GridSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=True),
             error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('feat_sel',
                                        RFE(estimator=LinearRegression(copy_X=True,
                                                                       fit_intercept=True,
                                                                       n_jobs=None,
                                                                       normalize=False),
                                            n_features_to_select=None, step=1,
                                            verbose=1)),
                                       ('reg',
                                        LinearRegression(copy_X=True,
                                                         fit_intercept=True,
                                                         n_jobs=None,
                                         

In [20]:
pd.DataFrame(gridcv.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_feat_sel__n_features_to_select,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.021210,0.000553,0.001171,0.000061,1,{'feat_sel__n_features_to_select': 1},0.524493,0.549792,0.550835,0.541661,0.012195,13
1,0.014878,0.001204,0.001056,0.000087,2,{'feat_sel__n_features_to_select': 2},0.650056,0.602361,0.688281,0.646907,0.035102,12
2,0.013581,0.000845,0.001144,0.000120,3,{'feat_sel__n_features_to_select': 3},0.659762,0.649197,0.735322,0.681370,0.038318,11
3,0.013719,0.000405,0.001020,0.000057,4,{'feat_sel__n_features_to_select': 4},0.660106,0.649424,0.735742,0.681700,0.038387,10
4,0.012320,0.000170,0.000926,0.000226,5,{'feat_sel__n_features_to_select': 5},0.707770,0.646046,0.760037,0.704626,0.046529,8
5,0.012507,0.001134,0.001094,0.000059,6,{'feat_sel__n_features_to_select': 6},0.699558,0.665311,0.736655,0.700505,0.029096,9
6,0.009921,0.000578,0.001152,0.000119,7,{'feat_sel__n_features_to_select': 7},0.709140,0.685349,0.745620,0.713359,0.024755,6
7,0.010904,0.002264,0.001237,0.000232,8,{'feat_sel__n_features_to_select': 8},0.718404,0.683344,0.757878,0.719871,0.030406,2
8,0.007862,0.000344,0.001072,0.000108,9,{'feat_sel__n_features_to_select': 9},0.716076,0.664957,0.752732,0.711268,0.035949,7
9,0.006534,0.000395,0.000939,0.000108,10,{'feat_sel__n_features_to_select': 10},0.725615,0.673008,0.751495,0.716730,0.032616,5


In [21]:
gridcv.best_estimator_

Pipeline(memory=None,
         steps=[('feat_sel',
                 RFE(estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                                n_jobs=None, normalize=False),
                     n_features_to_select=13, step=1, verbose=1)),
                ('reg',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbose=False)

In [22]:
y_pred = gridcv.predict(X_test)

In [23]:
from sklearn import metrics

metrics.r2_score(y_test, y_pred)

0.6753315195564333